In [ ]:
# import nltk
# nltk.download()

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import json

In [2]:
#read in csv
types = pd.read_csv("../mbti_1.csv")
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [3]:
types['posts'].iloc[0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

# EDA

In [4]:
types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [5]:
# check for null
types.isnull().sum()

type     0
posts    0
dtype: int64

In [6]:
# unique check 
types.nunique()

type       16
posts    8675
dtype: int64

In [7]:
types.type.value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [8]:
# add plots

# Preprocessing

In [9]:
# removing piping
types['posts']= types['posts'].str.replace('|',' ')
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw ...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired. That's another silly misconce...


In [10]:
# removing '
types['posts']= types['posts'].str.replace("'",'')
types.head()

,type,posts
0,INFJ,http://www.youtube.com/watch?v=qsXHcwe3krw h...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,Good one _____ https://www.youtube.com/watc...
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [11]:
# removing url's from posts
types['posts'] = types['posts'].str.replace('http\S+|www.\S+', '', case=False)
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter no...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,"Good one _____ Of course, to which I say..."
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [12]:
# change case to lower
types['posts'] = types['posts'].str.lower()
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter no...
1,ENTP,im finding the lack of me in these posts very ...
2,INTP,"good one _____ of course, to which i say..."
3,INTJ,"dear intp, i enjoyed our conversation the ot..."
4,ENTJ,youre fired. thats another silly misconcepti...


In [13]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
#remove punctuation from posts

def remove_punctuation(text):
    no_punctuation = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation

types['body_text_clean'] = types['posts'].apply(lambda x: remove_punctuation(x))


In [15]:
# pulling types from type column
mbti_types = types['type'].unique()

In [16]:
# types to list instead of array
mbti_list = mbti_types.tolist()

In [17]:
# lowercasing types
mbti_new = [x.lower() for x in mbti_list]

In [18]:
# remove references to mbti type in body_text_clean column

for item in mbti_new:
    types['body_text_clean'] = types['body_text_clean'].str.replace(item , "")

types.head()

,type,posts,body_text_clean
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...


In [19]:
# apply word_tokenize to all records
from nltk.tokenize import word_tokenize

types['tokenized'] = types['body_text_clean'].apply(word_tokenize)

types.head()

,type,posts,body_text_clean,tokenized
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce..."


In [20]:
# types['body_text_clean'].iloc[0]

In [23]:
# remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')


def stopword_removal(text):
    stop_words = [item for item in text if item not in stop]
    return stop_words

types['stopwords'] = types['tokenized'].apply(lambda x: stopword_removal(x))
types.head()

,type,posts,body_text_clean,tokenized,stopwords
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl...","[moments, sportscenter, top, ten, plays, prank..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po...","[im, finding, lack, posts, alarming, sex, bori..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ...","[good, one, course, say, know, thats, blessing..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth...","[dear, enjoyed, conversation, day, esoteric, g..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce...","[youre, fired, thats, another, silly, misconce..."


In [ ]:
# stemming

In [ ]:
# lemmatize